## **Laboratorio 4 - Data Science**
### *Mejorando el Análisis de Sentimientos con LSTM y Características Adicionales*
Stefano Aragoni, Carol Arévalo

----------

#### `Importación de Datos`
- Utilice el conjunto de datos IMDB proporcionado por Keras. pero esta vez, en lugar de utilizar sólo las 20.000 palabras más frecuentes,utilice las 50.000 palabras más frecuentes


#### `Pre-procesamiento`
- Secuencie y rellene las críticas para que todas tengan una longitud uniforme.
- De las críticas, extraiga características (features) adicionales, por ejemplo. la longitud de la crítica, la proporción de palabras positivas/negativas y cualquier otra que considere pueda ser útil.

#### `Modelo`
- Cree un modelo LSTM que acepte las características (features) adicionales junto con la secuencia de palabras.
- Intente usar una arquitectura más compleja, incorporando más capas LSTM, capas de Dropout para la regularización y tal vez alguna capa densamente conectada después de la LSTM. (ver también la referencia al final de este documento)

#### `Entrenamiento y Evaluación`
- Entrene su modelo con el conjunto de datos de entrenamiento y evalúe su desempeño con el conjunto de datos de prueba.

*Según Luis Furlán, el Jupyter Notebook cuenta como el informe en sí.*